# 把判決全文加到database

In [18]:
import os
import glob
import json
import sqlite3
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

In [13]:
con = sqlite3.connect('cases.db')
cur = con.cursor()

In [14]:
id_list_sql = """SELECT c28裁判書ID FROM Cases WHERE c28裁判書ID IS NOT NULL"""
verdict_id_list = cur.execute(id_list_sql).fetchall()

In [ ]:
for verdict_id in tqdm(verdict_id_list):
    try:
        with open(f'D:/all_verdict_temp/verdict_api/json_files/{verdict_id[0]}.json', 'r') as f:
            data = json.load(f)
            verdict = (data['JFULLX']['JFULLCONTENT'],)
            sql = f"""UPDATE Cases SET verdict_text = ? WHERE c28裁判書ID = '{verdict_id[0]}'"""
            cur.execute(sql, verdict)
            print(f"Successfully updated verdict text for {verdict_id[0]}")
    except FileNotFoundError:
        print("The specified JSON file does not exist.")
    except KeyError as e:
        print(f"KeyError: {e} not found in the JSON data.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [24]:

# Collect success and error messages
success_updates = []
errors = []

# Prepare SQL statement for batch execution
sql = "UPDATE Cases SET verdict_text = ? WHERE c28裁判書ID = ?"

# Collecting updates
updates = []

for verdict_id in tqdm(verdict_id_list):
    try:
        json_file_path = f'D:/all_verdict_temp/verdict_api/json_files/{verdict_id[0]}.json'
        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as f:
                data = json.load(f)
                verdict = data['JFULLX']['JFULLCONTENT']
                updates.append((verdict, verdict_id[0]))
                success_updates.append(verdict_id[0])
        else:
            errors.append(f"The specified JSON file does not exist for verdict_id {verdict_id[0]}.")
    except KeyError as e:
        errors.append(f"KeyError: {e} not found in the JSON data for verdict_id {verdict_id[0]}.")
    except sqlite3.Error as e:
        errors.append(f"Database error: {e} for verdict_id {verdict_id[0]}.")
    except Exception as e:
        errors.append(f"An error occurred: {e} for verdict_id {verdict_id[0]}.")

# Execute batch update
try:
    cur.executemany(sql, updates)
    con.commit()
except sqlite3.Error as e:
    errors.append(f"Database error during batch update: {e}.")

# Print summary of operations
print(f"Successfully updated verdict text for {len(success_updates)} verdicts.")
if errors:
    print(f"Errors encountered: {len(errors)}")
    for error in errors:
        print(error)

# Close the database connection
con.close()


  0%|          | 0/1665590 [00:00<?, ?it/s]

In [16]:
with open(f'D:/all_verdict_temp/verdict_api/json_files/{verdict_id_list[66699][0]}.json', 'r') as f:
        data = json.load(f)

In [17]:
(data['JFULLX']['JFULLCONTENT'],)

('臺灣高雄地方法院刑事簡易判決\u3000\u3000\u3000\u3000 102年度簡字第2430號\r\n聲\u3000請\u3000人\u3000臺灣高雄地方法院檢察署檢察官\r\n被\u3000\u3000\u3000告\u3000任德慶\r\n上列被告因違反毒品危害防制條例案件，經檢察官聲請以簡易判\r\n決處刑（102年度毒偵字第931號），本院判決如下：\r\n    主      文\r\n任德慶施用第二級毒品，處有期徒刑貳月，如易科罰金，以新臺\r\n幣壹仟元折算壹日。\r\n    事實及理由\r\n一、犯罪事實：\r\n    任德慶前因施用毒品案件，經本院裁定送觀察、勒戒後，認\r\n    無繼續施用毒品之傾向，於民國99年7 月2 日執行完畢釋放\r\n    ，並經臺灣高雄地方法院檢察署檢察官以99年度毒偵字第18\r\n    13號為不起訴處分確定。詎其仍不知悔改，於觀察、勒戒執\r\n    行完畢釋放後5 年內，竟基於施用第二級毒品甲基安非他命\r\n    之犯意，於101 年12月13日7 時50分前某時，在其位於高雄\r\n    市○○區○○路○○巷○ 號2 樓之住處，以燒烤方式施用甲基\r\n    安非他命1 次。嗣於同日7 時50分，因另案為警拘提，復經\r\n    採集尿液送驗後，結果呈安非他命及甲基安非他命陽性反應\r\n    ，因而查悉上情。案經高雄市政府警察局刑事警察大隊報告\r\n    臺灣高雄地方法院檢察署檢察官偵查後聲請以簡易判決處刑\r\n    。\r\n二、證據部分：\r\n（一）被告任德慶於偵查中之自白。\r\n（二）正修科技大學超微量研究科技中心尿液檢驗報告1份。\r\n（三）高雄市政府警察局刑警大隊偵三隊七分隊偵辦毒品案尿液\r\n      採證代碼對照表冊1 份。\r\n三、按違反毒品危害防制條例第10條之罪，經觀察、勒戒後，認\r\n    無繼續施用毒品之傾向，應即釋放，並為不起訴處分或不付\r\n    審理之裁定；次按觀察、勒戒執行完畢釋放後，5 年內再犯\r\n    第10條之罪者，檢察官應依法追訴，民國93年1 月9 日修正\r\n    施行之毒品危害防制條例第20條第2 項及同法第23條第2 項\r\n    就此分別定有明文。查被告任德慶前因施用毒品案件

In [10]:
verdict_id_list[0][0]

'LCDM,101,訴,6,20130102,1'

# 用query新增太花時間，改用pandas處理

In [15]:
# 
sql_get_cases = """SELECT * FROM Cases"""
df = pd.read_sql(sql_get_cases, con)

In [21]:
def get_verdict_text(verdict_id):
    try:
        with open(f'D:/all_verdict_temp/verdict_api/json_files/{verdict_id}.json', 'r') as f:
            data = json.load(f)
            return data['JFULLX']['JFULLCONTENT']
    except FileNotFoundError:
        return None
    except KeyError as e:
        return None
    except Exception as e:
        return None

In [22]:
df['verdict_text'] = df['c28裁判書ID'].progress_apply(get_verdict_text)

  0%|          | 0/1665590 [00:00<?, ?it/s]

In [23]:
df.to_sql('Cases_verdict', con, if_exists='replace', index=False)

1665590

In [24]:
con.close()